In [54]:
# imports
import os
import io
import sys
from openai import OpenAI
import gradio as gr
import subprocess 
from IPython.display import Markdown, display

In [55]:
OLLAMA_BASE_URL = "http://localhost:11434/v1"
openai = OpenAI()
ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')  

In [56]:
MODEL="gpt-oss:20b"

In [57]:
models = ["gpt-oss:20b","llama3.2:latest","llama3.1:8b","gemma3:270m",]
clients ={"gpt-oss:20b":ollama, "llama3.2:latest":ollama, "llama3.1:8b":ollama, "gemma3:270m":ollama}

In [58]:
from system_info import retrieve_system_info
system_info = retrieve_system_info()
system_info

{'os': {'system': 'Windows',
  'arch': 'AMD64',
  'release': '11',
  'version': '10.0.26200',
  'kernel': '11',
  'distro': None,
  'wsl': False,
  'rosetta2_translated': False,
  'target_triple': 'mingw32'},
 'package_managers': ['winget'],
 'cpu': {'brand': 'AMD Ryzen 7 5800HS with Radeon Graphics',
  'cores_logical': 16,
  'cores_physical': 8,
  'simd': []},
 'toolchain': {'compilers': {'gcc': 'gcc.exe (MinGW.org GCC-6.3.0-1) 6.3.0',
   'g++': 'g++.exe (MinGW.org GCC-6.3.0-1) 6.3.0',
   'clang': '',
   'msvc_cl': ''},
  'build_tools': {'cmake': '', 'ninja': '', 'make': ''},
  'linkers': {'ld_lld': ''}}}

In [59]:
message = f"""
Here is a report of the system information for my computer.
I want to run a C++ compiler to compile a single C++ file called main.cpp and then execute it in the simplest way possible.
Please reply with whether I need to install any C++ compiler to do this. If so, please provide the simplest step by step instructions to do so.

If I'm already set up to compile C++ code, then I'd like to run something like this in Python to compile and execute the code:
```python
compile_command = # something here - to achieve the fastest possible runtime performance
compile_result = subprocess.run(compile_command, check=True, text=True, capture_output=True)
run_command = # something here
run_result = subprocess.run(run_command, check=True, text=True, capture_output=True)
return run_result.stdout
```
Please tell me exactly what I should use for the compile_command and run_command.

System information:
{system_info}
"""

In [61]:
response = ollama.chat.completions.create(model="gpt-oss:20b", messages=[{"role":"user","content":message}])
display(Markdown(response.choices[0].message.content))


**Do you need to install a compiler?**  
No – your machine already has **g++ (MinGW‑GCC 6.3.0)** on the PATH, so you can compile and run C++ straight away.  
If you want the *fastest possible* runtime or need newer language features/optimisations, you could upgrade to a newer MinGW‑SDK (e.g. MinGW‑W64 11.x) via **winget** or `choco`, but it’s not strictly required for a single file.

---

## 1. Quick compile‑and‑run with the existing g++

```python
import subprocess

# 1. Compile the source file to a Windows executable
#    * -O3   – aggressive optimisation
#    * -march=native – allow GCC to emit instructions that match your CPU
#    * -static-libgcc -static-libstdc++ – bundle the runtime into the exe (optional)
#    * -std=c++17 – you can bump to c++20/+ if you like
compile_cmd = [
    "g++",
    "-O3",
    "-march=native",
    "-std=c++17",
    "-o", "main.exe",
    "main.cpp"
]

# Run the compiler
subprocess.run(compile_cmd, check=True, text=True, capture_output=True)

# 2. Run the executable
run_cmd = ["main.exe"]

run_result = subprocess.run(
    run_cmd,
    check=True,
    text=True,
    capture_output=True
)

print(run_result.stdout)   # whatever main.cpp prints
```

### What each part does

| Item | Purpose |
|------|---------|
| `g++` | Invokes the GCC C++ compiler that’s already on your PATH. |
| `-O3` | Maximal optimisation for speed (costs a little compile time). |
| `-march=native` | Lets GCC generate code tuned for the exact CPU you’re running on (your Ryzen 7 5800HS). |
| `-std=c++17` | Uses a recent language standard; change to `c++20` or `c++23` if you need newer features. |
| `-o main.exe` | Name of the output binary. |
| `main.cpp` | The source file you want to build. |

> **Tip** – If you don’t want the runtime libraries linked statically, omit the `-static-libgcc -static-libstdc++` flags (they’re only included above as an optional optimisation for a single‑executable distribution).

---

## 2. If you prefer a newer compiler or MSVC

| Tool | How to install |
|------|----------------|
| **MinGW‑W64** (latest GCC) | `winget install -e --id=mingw-w64.mingw64` |
| **MSVC (cl.exe)** | Install “Desktop development with C++” from the Visual Studio Installer. After installation, launch **x64 Native Tools Command Prompt for VS** and `cl main.cpp` will do the job. |

The MSVC route gives you the *official* Windows compiler.  For a single file, the command is:

```bash
cl /EHsc /O2 /std:c++17 main.cpp
main.exe
```

---

### Summary

* **You are already set up.**  
  Use the `g++` snippet above – no installation required.

* **If you want the best performance or newer language features:**  
  Upgrade to MinGW‑W64 via winget (or install Visual Studio and use cl.exe).

Happy compiling!

In [76]:
compile_command = [
        "g++",
        "main.cpp",
        "-o",
        "main.exe"
]

run_command = ["main.exe"]


In [77]:
system_prompt = """
Your task is to convert Python code into high performance C++ code.
Respond only with C++ code. Do not provide any explanation other than occasional comments.
The C++ response needs to produce an identical output in the fastest possible time.
"""

def user_prompt_for(python):
    return f"""
Port this Python code to C++ with the fastest possible implementation that produces identical output in the least time.
The system information is:
{system_info}
Your response will be written to a file called main.cpp and then compiled and executed; the compilation command is:
{compile_command}
Respond only with C++ code.
Python code to port:

```python
{python}
```
"""

In [78]:

def messages_for(python):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(python)}
    ]


In [79]:

def write_output(cpp):
    with open("main.cpp", "w", encoding = "utf-8") as f:
        f.write(cpp)

In [80]:

def port(client, model, python):
    client = clients[model]
    response = client.chat.completions.create(model=model, messages=messages_for(python))
    reply = response.choices[0].message.content
    reply = reply.replace('```cpp','').replace('```','')
    write_output(reply)
    return reply

In [81]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(200_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [82]:
def run_python(code):
    globals = {"__builtins__": __builtins__}
    exec(code, globals)

In [69]:
run_python(pi)

Result: 3.141592656089
Execution Time: 31.792946 seconds


In [87]:
display(Markdown(port(ollama, MODEL, pi)))


#include <bits/stdc++.h>
using namespace std;

int main() {
    ios::sync_with_stdio(false);
    cin.tie(nullptr);

    const long long iterations = 200'000'000;
    const double param1 = 4.0;
    const double param2 = 1.0;

    auto start = chrono::steady_clock::now();

    double result = 1.0;
    for (long long i = 1; i <= iterations; ++i) {
        double j = i * param1 - param2;
        result -= 1.0 / j;
        j = i * param1 + param2;
        result += 1.0 / j;
    }
    result *= 4.0;

    auto end = chrono::steady_clock::now();
    double elapsed = chrono::duration_cast<chrono::duration<double>>(end - start).count();

    printf("Result: %.12f\nExecution Time: %.6f seconds\n", result, elapsed);
    return 0;
}


In [85]:
def compile_and_run():
    subprocess.run(compile_command, check=True, text=True, capture_output=True)
    print(subprocess.run(run_command, check=True, text=True, capture_output=True).stdout)
    print(subprocess.run(run_command, check=True, text=True, capture_output=True).stdout)
    print(subprocess.run(run_command, check=True, text=True, capture_output=True).stdout)
    print(subprocess.run(run_command, check=True, text=True, capture_output=True).stdout)

In [86]:

compile_and_run()

Result: 3.141592656089
Execution Time: 8.710422 seconds

Result: 3.141592656089
Execution Time: 6.893104 seconds

Result: 3.141592656089
Execution Time: 6.958230 seconds

Result: 3.141592656089
Execution Time: 6.945156 seconds

